In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import math
from tqdm import tqdm
import requests
from urllib.parse import urlparse
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
import datetime
from sklearn.preprocessing import quantile_transform
pd.options.mode.chained_assignment = None  # default='warn'

# Read Data

In [2]:
def file_reader(basedir, filename):
    if filename.endswith('.csv'):
        try:
            df = pd.read_csv(basedir + filename, header=15)
        except:
            df = pd.read_csv(basedir + filename, encoding='euc-kr', header=15)
    elif filename.endswith('.xlsx'):
        try:
            df = pd.read_excel(basedir + filename, header=16)
        except:
            df = pd.read_excel(basedir + filename, encoding='euc-kr', header=16)
    else:
        print('error')
    
    return df

In [3]:
def read_data():
    basedir = './국토교통부_실거래가_공개시스템/'
    filenames = os.listdir(basedir)
    
    apart_trade_files = [filename for filename in filenames if filename.startswith('아파트(매매)')]
    apart_lease_files = [filename for filename in filenames if filename.startswith('아파트(전월세)')]
    multiplex_trade_files = [filename for filename in filenames if filename.startswith('연립다세대(매매)')]
    multiplex_lease_files = [filename for filename in filenames if filename.startswith('연립다세대(전월세)')]
    officetel_trade_files = [filename for filename in filenames if filename.startswith('오피스텔(매매)')]
    officetel_lease_files = [filename for filename in filenames if filename.startswith('오피스텔(전월세)')]
    
    #
    apart_trade_dfs_list = []
    for filename in apart_trade_files:
        df = file_reader(basedir, filename)
        apart_trade_dfs_list.append(df)
    apart_trade_df = pd.concat(apart_trade_dfs_list).reset_index(drop=True)
    
    #
    apart_lease_dfs_list = []
    for filename in apart_lease_files:
        df = file_reader(basedir, filename)
        apart_lease_dfs_list.append(df)
    apart_lease_df = pd.concat(apart_lease_dfs_list).reset_index(drop=True)
    
    #
    multiplex_trade_dfs_list = []
    for filename in multiplex_trade_files:
        df = file_reader(basedir, filename)
        multiplex_trade_dfs_list.append(df)
    multiplex_trade_df = pd.concat(multiplex_trade_dfs_list).reset_index(drop=True)
    
    #
    multiplex_lease_dfs_list = []
    for filename in multiplex_lease_files:
        df = file_reader(basedir, filename)
        multiplex_lease_dfs_list.append(df)
    multiplex_lease_df = pd.concat(multiplex_lease_dfs_list).reset_index(drop=True)
    
    #
    officetel_trade_dfs_list = []
    for filename in officetel_trade_files:
        df = file_reader(basedir, filename)
        officetel_trade_dfs_list.append(df)
    officetel_trade_df = pd.concat(officetel_trade_dfs_list).reset_index(drop=True)
    
    #
    officetel_lease_dfs_list = []
    for filename in officetel_lease_files:
        df = file_reader(basedir, filename)
        officetel_lease_dfs_list.append(df)
    officetel_lease_df = pd.concat(officetel_lease_dfs_list).reset_index(drop=True)
    
    
    return apart_trade_df, apart_lease_df, multiplex_trade_df, multiplex_lease_df, officetel_trade_df, officetel_lease_df

In [4]:
%%time
apart_trade_df, apart_lease_df, multiplex_trade_df, multiplex_lease_df, officetel_trade_df, officetel_lease_df\
= read_data()

C:\Users\나\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:9: FutureWarning: Your version of xlrd is 1.2.0. In xlrd >= 2.0, only the xls format is supported. As a result, the openpyxl engine will be used if it is installed and the engine argument is not specified. Install openpyxl instead.
  if __name__ == '__main__':
C:\Users\나\AppData\Roaming\Python\Python37\site-packages\IPython\core\magics\execution.py:1321: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code, glob, local_ns)


Wall time: 4min 46s


# From previous EDA and Preprocessing

In [5]:
apart_lease_df.drop(apart_lease_df[apart_lease_df['전용면적(㎡)'].isna()].index, inplace=True)

In [6]:
multiplex_trade_df.loc[multiplex_trade_df[multiplex_trade_df['건축년도'].isna()].index, '건축년도'] = 1971
multiplex_lease_df.drop(multiplex_lease_df[multiplex_lease_df['층'].isna()].index, inplace=True)

In [7]:
# 모든 df 도로명 빈 칸 null 로 교체

def replace_empty_to_null(df):
    df = df.copy()
    
    empty_df = df[df['도로명'] == ' ']
    
    df.loc[empty_df.index, '도로명'] = np.nan
    
    return df

In [8]:
apart_trade_df = replace_empty_to_null(apart_trade_df)
apart_lease_df = replace_empty_to_null(apart_lease_df)
multiplex_trade_df = replace_empty_to_null(multiplex_trade_df)
multiplex_lease_df = replace_empty_to_null(multiplex_lease_df)
officetel_trade_df = replace_empty_to_null(officetel_trade_df)
officetel_lease_df = replace_empty_to_null(officetel_lease_df)

In [9]:
multiplex_lease_df.loc[multiplex_lease_df[multiplex_lease_df['건축년도'].isna()].index, '건축년도'] = 1971

In [10]:
# 건축년도가 null 인 row 들에 있는 주소들 중에서, 같은 주소지만 건축년도 값이 있는 주소는 null 을 채워넣고, 없는 주소는 df 에서 제거.
# 추가적으로, 한 주소지, 단지이름에 건축년도가 두 가지 이상 있는지도 조사함.

def bltyear_fill_and_remove_null(df):
    df = df.copy()
    
    df['temp_full_addr'] = df['시군구'] + df['번지'] + df['단지명']
    
    bltyear_null_df = df[df['건축년도'].isna()]
    
    unique_addrs = bltyear_null_df['temp_full_addr'].unique()
    
    concat_list = []
    for addr in unique_addrs:
        addr_df = df[df['temp_full_addr'] == addr]
        if addr_df['건축년도'].isna().sum() != addr_df.shape[0]:
            if addr_df['건축년도'].std() != 0:
                print('건축년도가 두 가지 이상인 주소가 있음.')
            
            addr_df['건축년도'].fillna(addr_df['건축년도'].mean(), inplace=True)
            
            concat_list.append(addr_df)
    
    df.drop(columns=['temp_full_addr'], inplace=True)
    
    if len(concat_list) > 0:
        concat_df = pd.concat(concat_list)
    else:
        return df
    
    #return concat_df
    
    df.loc[concat_df.index, '건축년도'] = concat_df['건축년도']
    
    bltyear_null_df = df[df['건축년도'].isna()]
    
    df.drop(bltyear_null_df.index, inplace=True)
        
    return df.reset_index(drop=True)

In [11]:
officetel_trade_df = bltyear_fill_and_remove_null(officetel_trade_df)
officetel_lease_df = bltyear_fill_and_remove_null(officetel_lease_df)

In [12]:
def trade_dfs_change_dtype(df):
    df = df.copy()
    
    df['거래금액(만원)'] = df['거래금액(만원)'].apply(str)
    df['거래금액(만원)'] = pd.to_numeric(df['거래금액(만원)'].str.replace(',', ''))
    df['층'] = df['층'].astype('int')
    df['건축년도'] = df['건축년도'].astype('int')
    
    return df

In [13]:
apart_trade_df = trade_dfs_change_dtype(apart_trade_df)
multiplex_trade_df = trade_dfs_change_dtype(multiplex_trade_df)
officetel_trade_df = trade_dfs_change_dtype(officetel_trade_df)

In [14]:
def lease_dfs_change_dtype(df):
    df = df.copy()
    
    df['보증금(만원)'] = df['보증금(만원)'].apply(str)
    df['월세(만원)'] = df['월세(만원)'].apply(str)
    df['보증금(만원)'] = pd.to_numeric(df['보증금(만원)'].str.replace(',', ''))
    df['월세(만원)'] = pd.to_numeric(df['월세(만원)'].str.replace(',', ''))
    df['층'] = df['층'].astype('int')
    df['건축년도'] = df['건축년도'].astype('int')
    
    return df

In [15]:
apart_lease_df = lease_dfs_change_dtype(apart_lease_df)
multiplex_lease_df = lease_dfs_change_dtype(multiplex_lease_df)
officetel_lease_df = lease_dfs_change_dtype(officetel_lease_df)

In [16]:
def price_per_area_and_year_month(df, mode='trade'):
    df = df.copy()
    
    if mode == 'trade':
        df['가격/면적'] = df['거래금액(만원)'] / df['전용면적(㎡)']
    elif mode == 'lease':
        df['가격/면적'] = df['보증금(만원)'] / df['전용면적(㎡)']
    
    df['year'] = pd.to_numeric(df['계약년월'].astype('str').str[:4])
    df['month'] = pd.to_numeric(df['계약년월'].astype('str').str[4:])
    
    return df   

In [17]:
apart_trade_df = price_per_area_and_year_month(apart_trade_df)
multiplex_trade_df = price_per_area_and_year_month(multiplex_trade_df)
officetel_trade_df = price_per_area_and_year_month(officetel_trade_df)

In [18]:
apart_lease_df = price_per_area_and_year_month(apart_lease_df, mode='lease')
multiplex_lease_df = price_per_area_and_year_month(multiplex_lease_df, mode='lease')
officetel_lease_df = price_per_area_and_year_month(officetel_lease_df, mode='lease')

In [19]:
def si_gu(df):
    df = df.copy()
    
    df['시'] = df['시군구'].str.split(' ').apply(lambda x: x[0])
    df['구'] = df['시군구'].str.split(' ').apply(lambda x: x[1])
    
    return df

In [20]:
apart_trade_df = si_gu(apart_trade_df)
apart_lease_df = si_gu(apart_lease_df)
multiplex_trade_df = si_gu(multiplex_trade_df)
multiplex_lease_df = si_gu(multiplex_lease_df)
officetel_trade_df = si_gu(officetel_trade_df)
officetel_lease_df = si_gu(officetel_lease_df)

In [21]:
def remove_zero_price(df, mode='trade'):
    df = df.copy()
    
    if mode == 'trade':
        df = df[df['거래금액(만원)'] != 0].reset_index(drop=True)
    elif mode == 'lease':
        df = df[df['보증금(만원)'] != 0].reset_index(drop=True)
    
    return df

In [22]:
apart_trade_df = remove_zero_price(apart_trade_df)
apart_lease_df = remove_zero_price(apart_lease_df, mode='lease')
multiplex_trade_df = remove_zero_price(multiplex_trade_df)
multiplex_lease_df = remove_zero_price(multiplex_lease_df, mode='lease')
officetel_trade_df = remove_zero_price(officetel_trade_df)
officetel_lease_df = remove_zero_price(officetel_lease_df, mode='lease')

In [23]:
def remove_zero_area(df):
    df = df.copy()
    
    df = df[df['전용면적(㎡)'] != 0].reset_index(drop=True)
    
    return df

In [30]:
apart_trade_df = remove_zero_area(apart_trade_df)
apart_lease_df = remove_zero_area(apart_lease_df)
multiplex_trade_df = remove_zero_area(multiplex_trade_df)
multiplex_lease_df = remove_zero_area(multiplex_lease_df)
officetel_trade_df = remove_zero_area(officetel_trade_df)
officetel_lease_df = remove_zero_area(officetel_lease_df)

In [33]:
def correct_lease_type(df):
    df = df.copy()
    
    wrong_df = df[(df['전월세구분'] == '월세') & (df['월세(만원)'] == 0)]
    
    df.loc[wrong_df.index, '전월세구분'] = '전세'
    
    wrong_df2 = df[(df['전월세구분'] == '전세') & (df['월세(만원)'] != 0)]
    
    df.loc[wrong_df2.index, '전월세구분'] = '월세'
    
    return df

In [34]:
apart_lease_df = correct_lease_type(apart_lease_df)
multiplex_lease_df = correct_lease_type(multiplex_lease_df)
officetel_lease_df = correct_lease_type(officetel_lease_df)

In [ ]:
'''apart_lease_deposit_only_df = apart_lease_df[apart_lease_df['전월세구분'] == '전세'].reset_index(drop=True)
apart_lease_monthly_pay_df = apart_lease_df[apart_lease_df['전월세구분'] == '월세'].reset_index(drop=True)
multiplex_lease_deposit_only_df = multiplex_lease_df[multiplex_lease_df['전월세구분'] == '전세'].reset_index(drop=True)
multiplex_lease_monthly_pay_df = multiplex_lease_df[multiplex_lease_df['전월세구분'] == '월세'].reset_index(drop=True)
officetel_lease_deposit_only_df = officetel_lease_df[officetel_lease_df['전월세구분'] == '전세'].reset_index(drop=True)
officetel_lease_monthly_pay_df = officetel_lease_df[officetel_lease_df['전월세구분'] == '월세'].reset_index(drop=True)'''

# EDA & Preprocessing 11

In [35]:
dfs_list = [apart_lease_df, multiplex_lease_df, officetel_lease_df]
for df in dfs_list:
    wrong_df = df[(df['전월세구분'] == '전세') & (df['월세(만원)'] != 0)]
    print(wrong_df.shape)

(0, 19)
(0, 19)
(0, 19)


In [29]:
wrong_df

,시군구,번지,본번,부번,단지명,전월세구분,전용면적(㎡),계약년월,계약일,보증금(만원),월세(만원),층,건축년도,도로명,가격/면적,year,month,시,구
5201,서울특별시 동대문구 장안동,373-6,373,6,황금오피스텔,전세,24.51,201110,25,4000,15,7,1997,장한로21길 11-11,163.198694,2011,10,서울특별시,동대문구
7183,서울특별시 송파구 잠실동,175-12,175,12,올림픽타워,전세,34.87,201102,21,10000,50,3,2004,올림픽로 88,286.779467,2011,2,서울특별시,송파구
51235,서울특별시 마포구 성산동,278-8,278,8,샬롬 오피스텔,전세,19.38,201403,8,7000,30,6,2003,성산로2길 31,361.197110,2014,3,서울특별시,마포구


# 이상치 제거 ver 2

In [36]:
apart_trade_df['target_log_transformed'] = apart_trade_df['거래금액(만원)'].apply(lambda x: np.log(x))
apart_lease_df['target_log_transformed'] = apart_lease_df['보증금(만원)'].apply(lambda x: np.log(x))
multiplex_trade_df['target_log_transformed'] = multiplex_trade_df['거래금액(만원)'].apply(lambda x: np.log(x))
multiplex_lease_df['target_log_transformed'] = multiplex_lease_df['보증금(만원)'].apply(lambda x: np.log(x))
officetel_trade_df['target_log_transformed'] = officetel_trade_df['거래금액(만원)'].apply(lambda x: np.log(x))
officetel_lease_df['target_log_transformed'] = officetel_lease_df['보증금(만원)'].apply(lambda x: np.log(x))

In [68]:
# Tukey의 기법을 사용하여 각 동 별 이상치 (outlier) 검색 후 제거 (가격)

def find_and_remove_Tukey_outliers_ver2_price_per_dong(df):
    df = df.copy()
    
    unique_dongs = df['시군구'].unique()
    
    count = 0
    dfs_list = []
    for dong in unique_dongs:
        dong_df = df[df['시군구'] == dong]
        
        year_val_c = dong_df['year'].value_counts()
        year_val_c_max = year_val_c.max()
        
        year_dfs_list = []
        for year in year_val_c.index:
            year_df = dong_df[dong_df['year'] == year]
            year_df_sample = year_df.sample(n=year_val_c_max, replace=True, random_state=42)
            year_dfs_list.append(year_df_sample)
            
        dong_concat_df = pd.concat(year_dfs_list)
    
        q1 = dong_concat_df['target_log_transformed'].quantile(0.25)
        q3 = dong_concat_df['target_log_transformed'].quantile(0.75)
        iqr = q3 - q1
        outlier_step = iqr * 1.5

        outlier_df = dong_df[(dong_df['target_log_transformed'] < q1 - outlier_step) | (dong_df['target_log_transformed'] > q3 + outlier_step)]

        dong_df.drop(outlier_df.index, inplace=True)
        
        dfs_list.append(dong_df)

        print('{}에서 {}개의 이상치를 찾아서 제거했습니다.'.format(dong, outlier_df.shape[0]))
        
        count += outlier_df.shape[0]
    
    print('총 {}개의 이상치를 찾아서 제거했습니다.'.format(count))
    
    concat_df = pd.concat(dfs_list).reset_index(drop=True)   
    
    return concat_df

In [69]:
prac_df = find_and_remove_Tukey_outliers_ver2_price_per_dong(apart_trade_df)

서울특별시 강남구 개포동에서 121개의 이상치를 찾아서 제거했습니다.
서울특별시 강남구 논현동에서 33개의 이상치를 찾아서 제거했습니다.
서울특별시 강남구 대치동에서 530개의 이상치를 찾아서 제거했습니다.
서울특별시 강남구 도곡동에서 300개의 이상치를 찾아서 제거했습니다.
서울특별시 강남구 삼성동에서 120개의 이상치를 찾아서 제거했습니다.
서울특별시 강남구 세곡동에서 3개의 이상치를 찾아서 제거했습니다.
서울특별시 강남구 수서동에서 11개의 이상치를 찾아서 제거했습니다.
서울특별시 강남구 신사동에서 53개의 이상치를 찾아서 제거했습니다.
서울특별시 강남구 압구정동에서 19개의 이상치를 찾아서 제거했습니다.
서울특별시 강남구 역삼동에서 0개의 이상치를 찾아서 제거했습니다.
서울특별시 강남구 일원동에서 64개의 이상치를 찾아서 제거했습니다.
서울특별시 강남구 청담동에서 62개의 이상치를 찾아서 제거했습니다.
서울특별시 강동구 강일동에서 109개의 이상치를 찾아서 제거했습니다.
서울특별시 강동구 고덕동에서 142개의 이상치를 찾아서 제거했습니다.
서울특별시 강동구 길동에서 607개의 이상치를 찾아서 제거했습니다.
서울특별시 강동구 둔촌동에서 18개의 이상치를 찾아서 제거했습니다.
서울특별시 강동구 명일동에서 25개의 이상치를 찾아서 제거했습니다.
서울특별시 강동구 상일동에서 208개의 이상치를 찾아서 제거했습니다.
서울특별시 강동구 성내동에서 188개의 이상치를 찾아서 제거했습니다.
서울특별시 강동구 암사동에서 9개의 이상치를 찾아서 제거했습니다.
서울특별시 강동구 천호동에서 245개의 이상치를 찾아서 제거했습니다.
서울특별시 강북구 미아동에서 193개의 이상치를 찾아서 제거했습니다.
서울특별시 강북구 번동에서 165개의 이상치를 찾아서 제거했습니다.
서울특별시 강북구 수유동에서 216개의 이상치를 찾아서 제거했습니다.
서울특별시 강북구 우이동에서 8개의 이상치를 찾아서 제거했습니다.
서울특별시 강서구 가양동에서 0개의 이상치를 찾아서 제거했습니다.
서울특별시

서울특별시 영등포구 영등포동7가에서 274개의 이상치를 찾아서 제거했습니다.
서울특별시 영등포구 영등포동8가에서 33개의 이상치를 찾아서 제거했습니다.
서울특별시 용산구 도원동에서 37개의 이상치를 찾아서 제거했습니다.
서울특별시 용산구 동빙고동에서 3개의 이상치를 찾아서 제거했습니다.
서울특별시 용산구 동자동에서 0개의 이상치를 찾아서 제거했습니다.
서울특별시 용산구 문배동에서 140개의 이상치를 찾아서 제거했습니다.
서울특별시 용산구 보광동에서 23개의 이상치를 찾아서 제거했습니다.
서울특별시 용산구 산천동에서 2개의 이상치를 찾아서 제거했습니다.
서울특별시 용산구 서빙고동에서 3개의 이상치를 찾아서 제거했습니다.
서울특별시 용산구 신계동에서 0개의 이상치를 찾아서 제거했습니다.
서울특별시 용산구 신창동에서 14개의 이상치를 찾아서 제거했습니다.
서울특별시 용산구 용문동에서 0개의 이상치를 찾아서 제거했습니다.
서울특별시 용산구 용산동2가에서 2개의 이상치를 찾아서 제거했습니다.
서울특별시 용산구 용산동5가에서 2개의 이상치를 찾아서 제거했습니다.
서울특별시 용산구 원효로1가에서 6개의 이상치를 찾아서 제거했습니다.
서울특별시 용산구 원효로4가에서 7개의 이상치를 찾아서 제거했습니다.
서울특별시 용산구 이촌동에서 49개의 이상치를 찾아서 제거했습니다.
서울특별시 용산구 이태원동에서 28개의 이상치를 찾아서 제거했습니다.
서울특별시 용산구 청암동에서 0개의 이상치를 찾아서 제거했습니다.
서울특별시 용산구 청파동1가에서 4개의 이상치를 찾아서 제거했습니다.
서울특별시 용산구 한강로1가에서 6개의 이상치를 찾아서 제거했습니다.
서울특별시 용산구 한강로2가에서 0개의 이상치를 찾아서 제거했습니다.
서울특별시 용산구 한강로3가에서 2개의 이상치를 찾아서 제거했습니다.
서울특별시 용산구 한남동에서 291개의 이상치를 찾아서 제거했습니다.
서울특별시 용산구 효창동에서 14개의 이상치를 찾아서 제거했습니다.
서울특별시 용산구 후암동에서 26개의 이상치를 찾아서 제거했습니

In [70]:
prac_df['거래금액(만원)'].describe()

count    1.147287e+06
mean     5.226836e+04
std      3.868994e+04
min      2.300000e+03
25%      2.860000e+04
50%      4.150000e+04
75%      6.278250e+04
max      5.700000e+05
Name: 거래금액(만원), dtype: float64

# 가격/면적 log

In [71]:
apart_trade_df['target/area_log_transformed'] = apart_trade_df['가격/면적'].apply(lambda x: np.log(x))
apart_lease_df['target/area_log_transformed'] = apart_lease_df['가격/면적'].apply(lambda x: np.log(x))
multiplex_trade_df['target/area_log_transformed'] = multiplex_trade_df['가격/면적'].apply(lambda x: np.log(x))
multiplex_lease_df['target/area_log_transformed'] = multiplex_lease_df['가격/면적'].apply(lambda x: np.log(x))
officetel_trade_df['target/area_log_transformed'] = officetel_trade_df['가격/면적'].apply(lambda x: np.log(x))
officetel_lease_df['target/area_log_transformed'] = officetel_lease_df['가격/면적'].apply(lambda x: np.log(x))

In [72]:
# Tukey의 기법을 사용하여 각 동 별 이상치 (outlier) 검색 후 제거 (가격)

def find_and_remove_Tukey_outliers_ver2_price_per_area_log_per_dong(df):
    df = df.copy()
    
    unique_dongs = df['시군구'].unique()
    
    count = 0
    dfs_list = []
    for dong in unique_dongs:
        dong_df = df[df['시군구'] == dong]
        
        year_val_c = dong_df['year'].value_counts()
        year_val_c_max = year_val_c.max()
        
        year_dfs_list = []
        for year in year_val_c.index:
            year_df = dong_df[dong_df['year'] == year]
            year_df_sample = year_df.sample(n=year_val_c_max, replace=True, random_state=42)
            year_dfs_list.append(year_df_sample)
            
        dong_concat_df = pd.concat(year_dfs_list)
    
        q1 = dong_concat_df['target/area_log_transformed'].quantile(0.25)
        q3 = dong_concat_df['target/area_log_transformed'].quantile(0.75)
        iqr = q3 - q1
        outlier_step = iqr * 1.5

        outlier_df = dong_df[(dong_df['target/area_log_transformed'] < q1 - outlier_step)\
                             | (dong_df['target/area_log_transformed'] > q3 + outlier_step)]

        dong_df.drop(outlier_df.index, inplace=True)
        
        dfs_list.append(dong_df)

        print('{}에서 {}개의 이상치를 찾아서 제거했습니다.'.format(dong, outlier_df.shape[0]))
        
        count += outlier_df.shape[0]
    
    print('총 {}개의 이상치를 찾아서 제거했습니다.'.format(count))
    
    concat_df = pd.concat(dfs_list).reset_index(drop=True)   
    
    return concat_df

In [73]:
prac_df = find_and_remove_Tukey_outliers_ver2_price_per_area_log_per_dong(apart_trade_df)

서울특별시 강남구 개포동에서 204개의 이상치를 찾아서 제거했습니다.
서울특별시 강남구 논현동에서 62개의 이상치를 찾아서 제거했습니다.
서울특별시 강남구 대치동에서 480개의 이상치를 찾아서 제거했습니다.
서울특별시 강남구 도곡동에서 123개의 이상치를 찾아서 제거했습니다.
서울특별시 강남구 삼성동에서 48개의 이상치를 찾아서 제거했습니다.
서울특별시 강남구 세곡동에서 0개의 이상치를 찾아서 제거했습니다.
서울특별시 강남구 수서동에서 52개의 이상치를 찾아서 제거했습니다.
서울특별시 강남구 신사동에서 6개의 이상치를 찾아서 제거했습니다.
서울특별시 강남구 압구정동에서 48개의 이상치를 찾아서 제거했습니다.
서울특별시 강남구 역삼동에서 6개의 이상치를 찾아서 제거했습니다.
서울특별시 강남구 일원동에서 204개의 이상치를 찾아서 제거했습니다.
서울특별시 강남구 청담동에서 65개의 이상치를 찾아서 제거했습니다.
서울특별시 강동구 강일동에서 5개의 이상치를 찾아서 제거했습니다.
서울특별시 강동구 고덕동에서 0개의 이상치를 찾아서 제거했습니다.
서울특별시 강동구 길동에서 171개의 이상치를 찾아서 제거했습니다.
서울특별시 강동구 둔촌동에서 46개의 이상치를 찾아서 제거했습니다.
서울특별시 강동구 명일동에서 241개의 이상치를 찾아서 제거했습니다.
서울특별시 강동구 상일동에서 89개의 이상치를 찾아서 제거했습니다.
서울특별시 강동구 성내동에서 177개의 이상치를 찾아서 제거했습니다.
서울특별시 강동구 암사동에서 516개의 이상치를 찾아서 제거했습니다.
서울특별시 강동구 천호동에서 459개의 이상치를 찾아서 제거했습니다.
서울특별시 강북구 미아동에서 552개의 이상치를 찾아서 제거했습니다.
서울특별시 강북구 번동에서 740개의 이상치를 찾아서 제거했습니다.
서울특별시 강북구 수유동에서 311개의 이상치를 찾아서 제거했습니다.
서울특별시 강북구 우이동에서 4개의 이상치를 찾아서 제거했습니다.
서울특별시 강서구 가양동에서 434개의 이상치를 찾아서 제거했습니다.
서울특별시 

서울특별시 영등포구 영등포동4가에서 12개의 이상치를 찾아서 제거했습니다.
서울특별시 영등포구 영등포동5가에서 23개의 이상치를 찾아서 제거했습니다.
서울특별시 영등포구 영등포동7가에서 114개의 이상치를 찾아서 제거했습니다.
서울특별시 영등포구 영등포동8가에서 24개의 이상치를 찾아서 제거했습니다.
서울특별시 용산구 도원동에서 179개의 이상치를 찾아서 제거했습니다.
서울특별시 용산구 동빙고동에서 3개의 이상치를 찾아서 제거했습니다.
서울특별시 용산구 동자동에서 12개의 이상치를 찾아서 제거했습니다.
서울특별시 용산구 문배동에서 119개의 이상치를 찾아서 제거했습니다.
서울특별시 용산구 보광동에서 23개의 이상치를 찾아서 제거했습니다.
서울특별시 용산구 산천동에서 121개의 이상치를 찾아서 제거했습니다.
서울특별시 용산구 서빙고동에서 53개의 이상치를 찾아서 제거했습니다.
서울특별시 용산구 신계동에서 4개의 이상치를 찾아서 제거했습니다.
서울특별시 용산구 신창동에서 25개의 이상치를 찾아서 제거했습니다.
서울특별시 용산구 용문동에서 0개의 이상치를 찾아서 제거했습니다.
서울특별시 용산구 용산동2가에서 3개의 이상치를 찾아서 제거했습니다.
서울특별시 용산구 용산동5가에서 0개의 이상치를 찾아서 제거했습니다.
서울특별시 용산구 원효로1가에서 0개의 이상치를 찾아서 제거했습니다.
서울특별시 용산구 원효로4가에서 14개의 이상치를 찾아서 제거했습니다.
서울특별시 용산구 이촌동에서 414개의 이상치를 찾아서 제거했습니다.
서울특별시 용산구 이태원동에서 31개의 이상치를 찾아서 제거했습니다.
서울특별시 용산구 청암동에서 2개의 이상치를 찾아서 제거했습니다.
서울특별시 용산구 청파동1가에서 6개의 이상치를 찾아서 제거했습니다.
서울특별시 용산구 한강로1가에서 36개의 이상치를 찾아서 제거했습니다.
서울특별시 용산구 한강로2가에서 30개의 이상치를 찾아서 제거했습니다.
서울특별시 용산구 한강로3가에서 28개의 이상치를 찾아서 제거했습니다.
서울특별시 용산구 한남동에서 11

In [74]:
prac_df['거래금액(만원)'].describe()

count    1.118603e+06
mean     5.208338e+04
std      3.965031e+04
min      3.400000e+03
25%      2.850000e+04
50%      4.100000e+04
75%      6.180000e+04
max      8.400000e+05
Name: 거래금액(만원), dtype: float64